# Mean and Rotation Extraction Notebook
This notebook will get mean and rotation of each box
And will use them as the state


In [16]:
import cv2
import glob
import numpy as np
import math
import os
import pickle

from cv2 import aruco
from tqdm import tqdm

from unitree_legged_msgs.msg import HighCmd

from contrastive_learning.tests.animate_markers import AnimateMarkers
from contrastive_learning.tests.animate_rvec_tvec import AnimateRvecTvec
from contrastive_learning.datasets.preprocess import get_rvec_tvec, dump_rvec_tvec, dump_pos_corners
from contrastive_learning.tests.plotting import plot_actions

CAMERA_INTRINSICS = np.array([[612.82019043,   0.        , 322.14050293],
                              [  0.        , 611.48303223, 247.9083252 ],
                              [  0.        ,   0.        ,   1.        ]])

## Get the data and find mean and rotation
Following cells will get the dumped corners and find the mean and the rotation of each box. All of this will be added to `preprocess.py` script to be used in actual training framework.

In [3]:
def get_smth_corners(root):
    with open(os.path.join(root, 'smoothened_corners.npy'), 'rb') as f:
        smth_corners = np.load(f)

    return smth_corners

In [4]:
def get_roots(data_root):
    data_dirs = glob.glob(f'{data_root}/*')
    roots = []
    for root in data_dirs:
        if os.path.isdir(root):
            roots.append(root)
    roots = sorted(roots)
    print('roots: {}'.format(roots))
    return roots

In [5]:
roots = get_roots(
    data_root='/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/'
)

roots: ['/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_10', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_12', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_13', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_14', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_16', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_18', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_2', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_20', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_2_demos/train_demos/box_marker_22', '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_o

In [6]:
smth_corners = get_smth_corners(root=roots[0])

In [7]:
def get_mean_and_rot(corner): # corner (4,2)
    mean = np.mean(corner, axis=0)
    diff = corner[0] - corner[1] # Just get two points
    rot = np.arctan2(diff[0], diff[1])
    mean_rot = np.concatenate((mean, [rot]), axis=-1)
    return mean_rot


In [8]:

def dump_pos_mean_rot(root, frame_interval=1):
    with open(os.path.join(root, 'pos_corners_fi_{}.pickle'.format(frame_interval)), 'rb') as f:
        pos_corners = pickle.load(f)

    pos_mean_rot = [] # Will have [mean_box_x, mean_box_y, box_rot, mean_dog_x, mean_dog_y, dog_rot]
    # Get the mean of each corner and find the rotation of the box
    for i in range(len(pos_corners)):
        curr_box, curr_dog = get_mean_and_rot(pos_corners[i][0][:4]), get_mean_and_rot(pos_corners[i][0][4:])
        next_box, next_dog = get_mean_and_rot(pos_corners[i][1][:4]), get_mean_and_rot(pos_corners[i][1][4:])
        action = pos_corners[i][2]

        pos_mean_rot.append((
            np.concatenate((curr_box, curr_dog)),
            np.concatenate((next_box, next_dog)),
            action
        ))
    
    with open(os.path.join(root, f'pos_mean_rot_fi_{frame_interval}.pickle'), 'wb') as f:
        pickle.dump(pos_mean_rot)

    return pos_mean_rot


In [9]:
pos_mean_rot = dump_pos_mean_rot(roots[0])

## Plot the extracted data 
Below will be the code to extract this new state representation.
All of this plotting will be added to `plotting.py` script.

In [17]:
import math 
import matplotlib.pyplot as plt

In [47]:
from typing import Tuple


def plot_mean_rot(ax, curr_pos, use_img=False, img=None, use_frame_axis=False, frame_axis=None, plot_action=False, actions=None, color_scheme=1):
    # actions: [action, pred_action]
    # curr_pos.shape: (6) [box_mean_x, box_mean_y, box_rot, dog_mean_x, dog_mean_y, dog_rot]

    img_shape = (720, 1280, 3)
    blank_image = np.ones(img_shape, np.uint8) * 255
    if use_img == False: # use img is when two plots are drawn on top of each other
        img = ax.imshow(blank_image.copy())

    if color_scheme == 1: # Actual corners - blue ones
        box_color = (0,0,255)
        dog_color = (0,0,153)
    else: # Predicted corners - predicted green ones
        box_color = (102,204,0)
        dog_color = (51,102,0)

    # Plot the rotation and the mean
    for j in range(2):
        curr_obs_pos = curr_pos[j*3:(j+1)*3]
        curr_mean = (int(curr_obs_pos[0]), int(curr_obs_pos[1]))
        curr_rot = curr_obs_pos[2]

        line_len = 50
        curr_end_point = (int(curr_mean[0] + np.cos(curr_rot) * line_len),
                          int(curr_mean[1] - np.sin(curr_rot) * line_len)) # (y starts from top left corner)

        if j == 0: # Show the box position
            if use_frame_axis:
                frame_axis = cv2.line(frame_axis.copy(), curr_mean, curr_end_point,
                                      color=box_color, thickness=3)
            else:
                frame_axis = cv2.line(blank_image.copy(), curr_mean, curr_end_point,
                                      color=box_color, thickness=3)

        else: # We will already have a frame axis given - show the dog position
            frame_axis = cv2.line(frame_axis.copy(), curr_mean, curr_end_point,
                                  color=dog_color, thickness=3)

    if plot_action:
        frame_axis = plot_actions(actions, frame_axis)

    img.set_array(frame_axis)
    ax.plot()

    return img, frame_axis


In [48]:
curr_pos_mean_rot = pos_mean_rot[0]

In [49]:
print(curr_pos_mean_rot)

(array([222.        , 356.25      ,  -2.94419709,  51.25      ,
       592.75      ,  -1.81096733]), array([222.        , 356.25      ,  -2.94419709,  65.75      ,
       569.5       ,  -1.77619172]), (0.15000000596046448, 0.0))


In [50]:

ncols = 10
nrows = 5
# print(N, ncols)
fig, axs = plt.subplots(figsize=(ncols*10,nrows*10), nrows=nrows, ncols=ncols) # Draw the predicted action

for i in range(50):
    axs_row = int(i / ncols)
    axs_col = int(i % ncols)

    curr_pos_mean_rot = pos_mean_rot[i]

    _, frame_axis = plot_mean_rot(axs[axs_row, axs_col], curr_pos_mean_rot[0], color_scheme=1)
    plot_mean_rot(axs[axs_row, axs_col], curr_pos_mean_rot[1], use_frame_axis=True, frame_axis=frame_axis, color_scheme=2)

plt.savefig('plot_mean_rot_ex.png')